# NFL Big Data Bowl 2024 - Tackling

Introduction

## Cleaning

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
tracking = []
for i in range(1, 10):
    tracking.append(pd.read_csv(f"data/tracking_week_{i}.csv"))
tracking = pd.concat(tracking)
tracking.reset_index(drop=True, inplace=True)

In [4]:
from Cleaning import clean_games_data, clean_plays_data, clean_players_data, \
    clean_tackles_data, clean_tracking_data
games = clean_games_data(pd.read_csv("data/games.csv"))
players = clean_players_data(pd.read_csv("data/players.csv"))
plays = clean_plays_data(pd.read_csv("data/plays.csv"))
tackles = clean_tackles_data(pd.read_csv("data/tackles.csv"))
tracking = clean_tracking_data(tracking)

Games data has been cleaned and memory has been reduced by 2.65625 bytes.
Players data has been cleaned and memory has been reduced by 13.1484375 bytes.
Plays data has been cleaned and memory has been reduced by 1041.80859375 bytes.
Tackles data has been cleaned and memory has been reduced by 476.4921875 bytes.
Tracking data has been cleaned and memory has been reduced by 476070.234375 bytes.


In [5]:
from Cleaning import check_for_snap, check_for_end
full_plays = check_for_snap(plays, tracking)
full_plays = check_for_end(full_plays, tracking)

  1%|▋                                          | 2/136 [00:02<03:00,  1.35s/it]

gameId = 2022091103 playId = 4175


  2%|▉                                          | 3/136 [00:03<02:46,  1.25s/it]

gameId = 2022091111 playId = 770


  7%|██▊                                        | 9/136 [00:11<02:32,  1.20s/it]

...

gameId = 2022091809 playId = 98


100%|█████████████████████████████████████████| 136/136 [02:47<00:00,  1.23s/it]


Removed 57 plays that do not have tracking at the snap of the ball.


100%|█████████████████████████████████████████| 136/136 [02:47<00:00,  1.23s/it]

Removed 136 plays that do not have tracking for the end of the play.


## Feature Engineering

In [9]:
# Make all the plays go from left to right to remove the variablilty it may cause
from Preprocessing import all_plays_left_to_right
from Preprocessing import create_acceleration_vectors, create_velocity_vectors, create_player_influence, \
    create_distance_to_ball

full_plays, full_tracking = all_plays_left_to_right(plays, tracking)

full_tracking = create_acceleration_vectors(full_tracking)
full_tracking = create_velocity_vectors(full_tracking)
full_tracking = create_player_influence(full_tracking)
full_tracking = create_distance_to_ball(full_tracking)

KeyError: "['influence_degree_football'] not found in axis"

In [ ]:
from Visualizations import animate_play

animate_play(games, full_plays, full_tracking, gameId=2022090800, playId=56, acceleration=True, velocity=True)

In [8]:
full_plays.query('gameId == 2022090800 and playId == 56')

,gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,passLength,penaltyYards,prePenaltyPlayResult,playResult,playNullifiedByPenalty,absoluteYardlineNumber,offenseFormation,defendersInTheBox,passProbability,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2


In [ ]:
data = pd.merge(plays, tracking, on = ["gameId", "playId"])

Finding who made the tackle on this play

Show the movement of the players during the play

In [ ]:
# Make a more customized version in plotly

In [ ]:
exampleFrame = data.query('gameId == 2022091103 and '
           'playId == 3126 and '
           'frameId == 20')

In [ ]:
from Visualizations import animate_play
animate_play(games, full_plays, full_tracking, gameId=2022091103, playId=3126, velocity=True)

In [ ]:
play = data[(data['gameId'] == 2022090800) & (data['playId'] == 80)]

In [ ]:
playerData = data[(data['gameId'] == 2022090800) & (data['playId'] == 56) & (data['nflId'] == 35472)]

Find the relationship between the player and the ball at all times.

In [ ]:
football = play[play['club'] == 'football']

footballAndPlayer = pd.merge(football, play, on = ['gameId', 'playId', 'ballCarrierId', 'ballCarrierDisplayName',
       'playDescription', 'quarter', 'down', 'yardsToGo', 'possessionTeam',
       'defensiveTeam', 'yardlineSide', 'yardlineNumber', 'gameClock',
       'preSnapHomeScore', 'preSnapVisitorScore', 'passResult', 'passLength',
       'penaltyYards', 'prePenaltyPlayResult', 'playResult',
       'playNullifiedByPenalty', 'absoluteYardlineNumber', 'offenseFormation',
       'defendersInTheBox', 'passProbability', 'preSnapHomeTeamWinProbability',
       'preSnapVisitorTeamWinProbability', 'homeTeamWinProbabilityAdded',
       'visitorTeamWinProbilityAdded', 'expectedPoints', 'expectedPointsAdded',
       'foulName1', 'foulName2', 'foulNFLId1', 'foulNFLId2',
        'frameId', 'time',
       'playDirection','event'], suffixes=('_football', '_player'))

In [ ]:
footballAndPlayer

In [ ]:
footballAndPlayer['distanceFromBall'] = np.sqrt((footballAndPlayer['x_x'] - footballAndPlayer['x_y'])**2 + (footballAndPlayer['y_x'] - footballAndPlayer['y_y'])**2)

Find the distance between the players and the ball at all times

In [ ]:
defense = footballAndPlayer[footballAndPlayer['club_y'] == footballAndPlayer.iloc[0]['defensiveTeam']]

Plot the distance between each player and the ball at all times during the play

In [ ]:
playersInGame = set(defense['displayName_y'])
for index, player in enumerate(playersInGame):
    plt.plot(defense[defense['displayName_y'] == player]['frameId'], defense[defense['displayName_y'] == player]['distanceFromBall'])
plt.legend(playersInGame)
plt.axhline(0)
plt.axvline(defense[defense['event'] == "tackle"]['frameId'].iloc[0])
plt.show()

Plot the speed of the player at all times

In [ ]:
playersInGame = set(defense['displayName_y'])
for index, player in enumerate(playersInGame):
    plt.plot(defense[defense['displayName_y'] == player]['frameId'], defense[defense['displayName_y'] == player]['s_y'])
plt.legend(playersInGame)
plt.axvline(defense[defense['event'] == "tackle"]['frameId'].iloc[0])
plt.show()

Plot the acceleration of the player at all times

In [ ]:
playersInGame = set(defense['displayName_y'])
for index, player in enumerate(playersInGame):
    plt.plot(defense[defense['displayName_y'] == player]['frameId'], defense[defense['displayName_y'] == player]['a_y'])
plt.legend(playersInGame)
plt.axvline(defense[defense['event'] == "tackle"]['frameId'].iloc[0])
plt.show()

# Classify each tackle as open-field, sideline, in the hole, solo, or gang.

Criteria for each tackle:
- open field
    - The defender is 
    
ToDo:
Use a semi-supervised algorothm to classify all tackles
Use this classification to rank players on their ability to make these specific tackles
Create a reinforcemnt learning agent to make each type of tackle and see what the computer can do to make each tackle